In [1]:
%pip install ultralytics



   ---------------------------------------- 0.0/6.2 MB ? eta -:--:--
   -------- ------------------------------- 1.3/6.2 MB 6.1 MB/s eta 0:00:01
   --------------- ------------------------ 2.4/6.2 MB 5.8 MB/s eta 0:00:01
   ---------------------- ----------------- 3.4/6.2 MB 6.7 MB/s eta 0:00:01
   ---------------------------- ----------- 4.5/6.2 MB 6.2 MB/s eta 0:00:01
   ----------------------------------- ---- 5.5/6.2 MB 5.4 MB/s eta 0:00:01
   ---------------------------------------- 6.2/6.2 MB 5.4 MB/s eta 0:00:00
  Attempting uninstall: sympy
    Found existing installation: sympy 1.13.3
    Uninstalling sympy-1.13.3:
      Successfully uninstalled sympy-1.13.3
Note: you may need to restart the kernel to use updated packages.
